## Домашнее задание 7

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек: 

  2.1. Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
  
  2.2. Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

## Решение:

Установим и импортируем необходимые библиотеки.

In [186]:
!pip install natasha stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [187]:
import numpy as np
import pandas as pd
import re
import keras
import tensorflow as tf

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from razdel import tokenize

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, GlobalAvgPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import categorical_crossentropy
from keras.callbacks import EarlyStopping

from nltk.probability import FreqDist

Загрузим датасеты и проведем предобработку данных.

In [188]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [189]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(ss.text)[0].normal_form for ss in tokenize(txt) if ss.text not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

df_train.head()

,id,text,class
0,0,alisachachka уезжаааааааать ❤ тожена хотеть уе...,0
1,1,rt galyginvadim ребята девчата кино любовь зав...,1
2,2,rt artemklyushin ктоненавидеть пробка ретвит rt,0
3,3,rt epupybobv хотеться котлета покиевск запретн...,1
4,4,karinekurganova yessboss босапоп есбосан боять...,1


In [190]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

tokens_filtered = [ss.text for ss in tokenize(train_corpus) if ss.text.isalpha()]

Зададим гиперпараметры, которые будут использоваться в рамках нашего обучения.

In [191]:
max_words = 1000
max_len = 140
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

Отберем наиболее часто встречающиеся токены.

In [192]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['rt',
 'd',
 'хотеть',
 'знать',
 'httptco',
 'ян',
 'мочь',
 'новый',
 'любить',
 'завтра']

Сфомируем словарь, в котором поставим в соответствие токены и индексы.

In [193]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [194]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = tokenize(text.lower())
    tokens_filtered = [ss.text for ss in tokens if ss.text.isalpha()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]


# Текст в индексы токенов
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [195]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 601,   3, 611], dtype=int32)

Подготовим данные для обучения нейронной сети.

In [196]:
num_classes = 2
y_train = tf.keras.utils.to_categorical(df_train["class"], num_classes)
y_val = tf.keras.utils.to_categorical(df_val["class"], num_classes)

Построим модель, зададим параметры оптимизатора и обучим ее.

In [197]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=300, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(40))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 140, 300)          300000    
                                                                 
 conv1d_22 (Conv1D)          (None, 136, 128)          192128    
                                                                 
 activation_51 (Activation)  (None, 136, 128)          0         
                                                                 
 global_max_pooling1d_13 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_44 (Dense)            (None, 40)                5160      
                                                                 
 activation_52 (Activation)  (None, 40)                0         
                                                     

In [198]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [199]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 654s 2s/step - loss: 0.5644 - accuracy: 0.6877 - val_loss: 0.5431 - val_accuracy: 0.7044
Epoch 2/20
319/319 [==============================] - 652s 2s/step - loss: 0.5320 - accuracy: 0.7131 - val_loss: 0.5421 - val_accuracy: 0.7049


Оценим качество построенной модели на валидационной выборке.

In [200]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])т

45/45 [==============================] - 22s 479ms/step - loss: 0.5481 - accuracy: 0.6989


Test score: 0.548051118850708
Test accuracy: 0.6989375352859497


В рамках построенной выше модели слой эмбеддинга инициализировался по умолчанию. 

Попробуем теперь использовать ту же структуру сети, но инициализировать этот слой с помощью набора предобученных эмбеддингов `ruwikiruscorpora_upos_cbow_300_10_2021` (взято отсюда - https://rusvectores.org/en/models/).

In [201]:
!wget http://vectors.nlpl.eu/repository/20/220.zip
!unzip -q 220.zip

--2022-09-04 18:58:13--  http://vectors.nlpl.eu/repository/20/220.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638171816 (609M) [application/zip]
Saving to: ‘220.zip.1’

220.zip.1           100%[===================>] 608.61M  18.3MB/s    in 35s     

2022-09-04 18:58:48 (17.5 MB/s) - ‘220.zip.1’ saved [638171816/638171816]

replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Создадим словарь с эмбеддингами из файла.

In [202]:
embeddings_index = {}
i = 0
ids = []
with open('model.txt') as f:
    for line in f:
      i += 1
      word, coefs = line.split(maxsplit=1)
      coefs = np.fromstring(coefs, "f", sep=" ")
      if i > 1:
        text, sem = word.split('_')
        embeddings_index[text] = coefs
        ids.append(text)


print(f'Добавлено {len(embeddings_index)} векторов.')

Добавлено 205707 векторов.


Создадим матрицу с эмбеддингами на основе нашего словаря сопоставлений токенов и индексов.

In [203]:
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

print(f'Добавлены эмбеддинги для {hits} слов ({misses} слов не найдено).')


Добавлены эмбеддинги для 867 слов (132 слов не найдено).


Теперь обучим модель.

In [204]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=300, 
                    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                    # trainable=False,
                    input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(40))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 140, 300)          300000    
                                                                 
 conv1d_23 (Conv1D)          (None, 136, 128)          192128    
                                                                 
 activation_55 (Activation)  (None, 136, 128)          0         
                                                                 
 global_max_pooling1d_14 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_47 (Dense)            (None, 40)                5160      
                                                                 
 activation_56 (Activation)  (None, 40)                0         
                                                     

In [205]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [206]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 655s 2s/step - loss: 0.5958 - accuracy: 0.6621 - val_loss: 0.5554 - val_accuracy: 0.6956
Epoch 2/20
319/319 [==============================] - 664s 2s/step - loss: 0.5370 - accuracy: 0.7103 - val_loss: 0.5490 - val_accuracy: 0.7000


Посмотрим на качество на валидационной выборке.

In [208]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 23s 504ms/step - loss: 0.5565 - accuracy: 0.6964


Test score: 0.55646812915802
Test accuracy: 0.696424663066864


Получается, что инициализация матрици предобученными эмбеддингами не помогла добиться более высокого качества модели. Это несколько контринтуитивно, на мой взгляд. 

Можно посмотреть какой результат получится, если зафиксировать эмбеддинги из скачанного датасета и не обучать их дополнительно. 

In [209]:
model = Sequential()
model.add(Embedding(input_dim=max_words, 
                    output_dim=300, 
                    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
                    trainable=False,
                    input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(40))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 140, 300)          300000    
                                                                 
 conv1d_24 (Conv1D)          (None, 136, 128)          192128    
                                                                 
 activation_59 (Activation)  (None, 136, 128)          0         
                                                                 
 global_max_pooling1d_15 (Gl  (None, 128)              0         
 obalMaxPooling1D)                                               
                                                                 
 dense_50 (Dense)            (None, 40)                5160      
                                                                 
 activation_60 (Activation)  (None, 40)                0         
                                                     

In [210]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [211]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
319/319 [==============================] - 414s 1s/step - loss: 0.6392 - accuracy: 0.6279 - val_loss: 0.6205 - val_accuracy: 0.6432
Epoch 2/20
319/319 [==============================] - 408s 1s/step - loss: 0.5988 - accuracy: 0.6665 - val_loss: 0.6145 - val_accuracy: 0.6547


In [212]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 22s 484ms/step - loss: 0.6145 - accuracy: 0.6545


Test score: 0.6145442724227905
Test accuracy: 0.6544548869132996


Вполне ожидаемо качество такой модели значительно хуже. Отчасти это связано и с тем, что не для всех слов в нашем датасете нашлись предобученные эмбеддинги в скачанном наборе.